In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
import json
import time
import re


In [ ]:
def df_lambda_replace_characters(df, column_name, new_column, dictionary):
    df[new_column] = df[column_name].apply(lambda x: replace_words(x, dictionary))       
    return df

In [ ]:
def replace_words(strtext, dictionary):
    pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in dictionary.keys()) + r')(?!\w)')
    result = pattern.sub(lambda x: dictionary[x.group()], strtext)
    return result

In [ ]:
with open("../replacement_dictionary_ordered.json") as json_file:
    replacement_dictionary_2 = json.load(json_file)

In [ ]:
states = ['TX', 'KS', 'AZ', 'TN', 'FL', 'GA', 'OK', 'UT', 'ME', 'CA', 'MS',
       'WA', 'OH', 'MA', 'AL', 'NC', 'CO', 'MO', 'KY', 'NE', 'NY',
       'VA', 'MD', 'HI', 'WV', 'MN', 'AK', 'IN', 'PA', 'SC', 'LA', 'NJ',
       'RI', 'WY']

In [ ]:
state = states[0]
noun_file = 'new_avm_df__full_data_state_'+state+'_noun_sentences.fea'
noun_sentence_df_T_ri_filtered = pd.read_feather(noun_file)

noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered[noun_sentence_df_T_ri_filtered['count'] >\
                                                                noun_sentence_df_T_ri_filtered['count'].max()*0.005]

noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.loc[:,['noun_sentence', 'count',
                                                                      'AVM_Error_Indicators']]




In [ ]:
len(noun_sentence_df_T_ri_filtered)

In [ ]:
column = 'AVM_Error_Indicators'
nelements = (len(noun_sentence_df_T_ri_filtered[column].values[0]))
list_tmp = [str(i) for i in range(nelements)] 
column_names = [column + '_' + str(i) for i in list_tmp] 
print(column_names)
split_df = pd.DataFrame(noun_sentence_df_T_ri_filtered[column].tolist(), columns=column_names, 
                        index=noun_sentence_df_T_ri_filtered.index)
noun_sentence_df_T_ri_filtered = pd.concat([noun_sentence_df_T_ri_filtered, split_df], axis=1)
noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.drop(columns = [column])

noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.loc[:,['noun_sentence','count',
                                                                       'AVM_Error_Indicators_0','AVM_Error_Indicators_1']]
noun_sentence_df_T_ri_filtered = df_lambda_replace_characters(noun_sentence_df_T_ri_filtered,  'noun_sentence', 
                                                   'noun_sentence_corrections', replacement_dictionary_2)
noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.loc[:,['noun_sentence_corrections','count',
                                                                       'AVM_Error_Indicators_0','AVM_Error_Indicators_1']]
noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.add_suffix('_'+state)


In [ ]:
noun_sentence_df_T_ri_filtered_full = noun_sentence_df_T_ri_filtered.copy(deep=True).\
                                rename(columns={'noun_sentence_corrections_'+state:'noun_sentence'})

In [ ]:
noun_sentence_df_T_ri_filtered_full = noun_sentence_df_T_ri_filtered_full.drop_duplicates(subset='noun_sentence')

In [ ]:
noun_sentence_df_T_ri_filtered_full = noun_sentence_df_T_ri_filtered_full.set_index('noun_sentence')

In [ ]:
noun_sentence_df_T_ri_filtered_full.sort_values('count_TX')

In [ ]:
noun_sentence_df_T_ri_filtered_full.loc['fridge stays']

In [ ]:
import math

In [ ]:
for state in states[1:]:
    print('full ',len(noun_sentence_df_T_ri_filtered_full))
    tic = time.time()
    print(state)
    noun_file = 'new_avm_df__full_data_state_'+state+'_noun_sentences.fea'
    noun_sentence_df_T_ri_filtered = pd.read_feather(noun_file)
    
    noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered[noun_sentence_df_T_ri_filtered['count'] >\
                                                                    math.ceil(noun_sentence_df_T_ri_filtered['count'].max()*0.005)]

    print(len(noun_sentence_df_T_ri_filtered))
    
    if len(noun_sentence_df_T_ri_filtered)>=1:
        noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.loc[:,['noun_sentence', 'count',
                                                                              'AVM_Error_Indicators']]

        column = 'AVM_Error_Indicators'
        nelements = (len(noun_sentence_df_T_ri_filtered[column].values[0]))
        list_tmp = [str(i) for i in range(nelements)] 
        column_names = [column + '_' + str(i) for i in list_tmp] 
        #print(column_names)
        split_df = pd.DataFrame(noun_sentence_df_T_ri_filtered[column].tolist(), columns=column_names, 
                                index=noun_sentence_df_T_ri_filtered.index)
        noun_sentence_df_T_ri_filtered = pd.concat([noun_sentence_df_T_ri_filtered, split_df], axis=1)
        noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.drop(columns = [column])

        noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.loc[:,['noun_sentence','count',
                                                                               'AVM_Error_Indicators_0','AVM_Error_Indicators_1']]
        noun_sentence_df_T_ri_filtered = df_lambda_replace_characters(noun_sentence_df_T_ri_filtered,  'noun_sentence', 
                                                           'noun_sentence_corrections', replacement_dictionary_2)
        noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.loc[:,['noun_sentence_corrections','count',
                                                                               'AVM_Error_Indicators_0','AVM_Error_Indicators_1']]
        noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.add_suffix('_'+state)

        noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.copy(deep=True).\
                                    rename(columns={'noun_sentence_corrections_'+state:'noun_sentence'})
        noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.drop_duplicates(subset='noun_sentence')
        noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.set_index('noun_sentence')

        print('filtered ', len(noun_sentence_df_T_ri_filtered))
        print('added ', len(noun_sentence_df_T_ri_filtered)+len(noun_sentence_df_T_ri_filtered_full))
        noun_sentence_df_T_ri_filtered_full = noun_sentence_df_T_ri_filtered_full.merge( noun_sentence_df_T_ri_filtered,
                                                                                    how='outer',left_index=True, right_index=True)
    
        
    print('full ',len(noun_sentence_df_T_ri_filtered_full))
    
    toc = time.time()
    print(toc - tic)
    print('')

In [ ]:
len(noun_sentence_df_T_ri_filtered_full)

In [ ]:
noun_sentence_df_T_ri_filtered_full

In [ ]:
noun_sentence_df_T_ri_filtered_full.head(20)

In [ ]:
len(noun_sentence_df_T_ri_filtered_full)

In [ ]:
file = 'noun_phrases_by_state_new_avm'

In [ ]:
noun_sentence_df_T_ri_filtered_full.to_csv(file+'.csv')
noun_sentence_df_T_ri_filtered_full.reset_index().to_feather(file+'.fea')

In [ ]:

import tensorflow as tf
import tensorflow_hub as hub

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"

model = hub.load(module_url)
print ("module %s loaded" % module_url)


In [ ]:
def embed(input):
    return model(input)

In [ ]:
noun_sentence_df_T_ri_filtered_full_embeddings = noun_sentence_df_T_ri_filtered_full.reset_index()

In [ ]:
noun_sentence_df_T_ri_filtered_full_embeddings

In [ ]:
column = 'noun_sentence'
tic = time.time()
new_column = column + '_embeddings'
noun_sentence_df_T_ri_filtered_full_embeddings[new_column] = noun_sentence_df_T_ri_filtered_full_embeddings[column].apply(lambda x: embed([x]).numpy()[0]) 
toc = time.time()
print(toc - tic)

In [ ]:
noun_sentence_df_T_ri_filtered_full_embeddings

In [ ]:
column = 'noun_sentence_embeddings'
nelements = (len(noun_sentence_df_T_ri_filtered_full_embeddings[column].values[0]))
list_tmp = [str(i) for i in range(nelements)] 
column_names = [column + '_' + str(i) for i in list_tmp] 
print(column_names)
split_df = pd.DataFrame(noun_sentence_df_T_ri_filtered_full_embeddings[column].tolist(), columns=column_names, 
                        index=noun_sentence_df_T_ri_filtered_full_embeddings.noun_sentence)

In [ ]:
split_df.T

In [ ]:
correlations_df = split_df.T.corr(method='pearson')

In [ ]:
correlations_df

In [ ]:
correlations_df.index

In [ ]:
list_of_lists = []
corr_threshold = 0.82

In [ ]:
for idd,sentence in enumerate(correlations_df.index):
    print(idd)
    #print(correlations_df.index[correlations_df.iloc[idd,:]>corr_threshold])
    list_of_lists.append(list(correlations_df.index[correlations_df.iloc[idd,:]>corr_threshold]))

In [ ]:
list_of_lists

In [ ]:
correlations_df_groups = correlations_df.copy(deep=True)

In [ ]:
groups = []
len_groups = []

In [ ]:
while len(correlations_df_groups) > 0:
    idd = correlations_df_groups.index[0]
    print(idd)
    print(len(correlations_df_groups))
    group = list(np.array(correlations_df_groups.loc[idd][correlations_df_groups.loc[idd]>0.5].index))
    groups.append(group)
    len_groups.append(len(group))
    correlations_df_groups = correlations_df_groups.drop(labels = group, axis = 1)
    correlations_df_groups = correlations_df_groups.drop(labels = group, axis = 0)

In [ ]:
len(groups)
np.array(len_groups).max()

In [ ]:
order_groups = np.array(len_groups).argsort()[::-1]

In [ ]:
groups[order_groups[0]]

In [ ]:
len(groups)

In [ ]:
from wordcloud import WordCloud

In [ ]:
wc = WordCloud(background_color="white", colormap="Dark2", max_font_size=150, random_state=42)

In [ ]:
plt.rcParams['figure.figsize']=[20,10]

In [ ]:
for idd, pos in enumerate(order_groups):
    list_g = groups[pos]
    print(idd+1, ' ',len(list_g))
    wc.generate(' '.join(list_g))
    plt.imshow(wc,interpolation="bilinear")
    plt.axis('off')
    plt.show()

In [ ]:
noun_sentence_df_T_ri_filtered

In [ ]:
for list_g in groups[0:1]:
    print(len(list_g))
    wc.generate(' '.join(list_g))
    plt.imshow(wc,interpolation="bilinear")
    plt.axis('off')
    plt.show()